In [88]:
%%time
import pandas as pd
import os
import numpy as np
import torch
import torch_geometric
from sklearn.preprocessing import LabelEncoder
import gc
from torch_geometric.data import Data

item_feature = pd.read_csv('../data/item_feature.csv')
test = pd.read_csv('../data/test.csv')
user_feature = pd.read_csv('../data/user_feature.csv')
train = pd.read_pickle('../data/sml_train.pkl')

Wall time: 1min 1s


In [89]:
%%time
train = train[train.behavior_type=='clk']
his = train[train.date<'2019-06-20'].drop_duplicates(subset=['user_id','item_id'])
now = train[train.date>='2019-06-20']
del train
train = his[['user_id','item_id']].merge(now[['user_id','item_id','behavior_type']],how='left')
del his,now
gc.collect()

Wall time: 1min 32s


## build model

In [79]:
%%time
train = pd.concat([train[train.behavior_type.isnull()==False],train[train.behavior_type.isnull()==True].sample(4000000)],axis=0)
train['behavior_type'] = train['behavior_type'].fillna(0)
train['behavior_type'] = train['behavior_type'].map({'clk':1})

u_enc,i_enc = LabelEncoder().fit(train['user_id']),LabelEncoder().fit(train['item_id'])
train['user_id'] = u_enc.transform(train['user_id'])
train['item_id'] = i_enc.transform(train['item_id'])+u_enc.classes_.shape[0]

edge_index = torch.tensor([train['user_id'].values,train['item_id'].values])
u = torch.tensor(train['user_id'].unique().reshape(-1,1))
i = torch.tensor(train['item_id'].unique().reshape(-1,1))
y = train['behavior_type'].values.reshape(-1,1)
data = Data(u=u,i=i,edge_index=edge_index,y=y)

Wall time: 11.3 s


In [84]:
y = train['behavior_type'].values.reshape(-1,1)
data = Data(u=u,i=i,edge_index=edge_index,y=y)

user_id                0
item_id                0
behavior_type    5176391
dtype: int64

In [80]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.u_emb = torch.nn.Embedding(len(u),5)
        self.i_emb = torch.nn.Embedding(len(i),5)
        self.conv1 = GCNConv(2*5,6)
        self.conv2 = GCNConv(6,2)
        
    def forward(self, data):
        u,i,edge_index = data.u,data.i,data.edge_index
        
        emb_u = self.u_emb(u[edge_index[0]]).view(-1,5)
        emb_i = self.i_emb(i[(edge_index[1]-u_enc.classes_.shape[0])]-u_enc.classes_.shape[0]).view(-1,5)

        x = torch.cat([emb_u,emb_i],dim=1)
        x = self.conv1(x,edge_index)
        x = F.relu(x)
        x = F.dropout(x,training=self.training)
        x = self.conv2(x,edge_index)
        
        return F.log_softmax(x,dim=1)
    


In [81]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out,data.y)
    loss.backward()
    optimizer.step
    if epoch%20==0:
        print(loss)

torch.Size([5176391, 5])
torch.Size([5176391, 5])


TypeError: 'int' object is not callable

In [83]:
y

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]])

tensor(2500616)

In [69]:
i.min()

tensor(810736)